# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [145]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [146]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [147]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


126


In [148]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

112


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [149]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [150]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [151]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [152]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query ="CREATE TABLE IF NOT EXISTS music_app_his_heard"
query = query + "(session_Id int, item_num_in_session int, artist text, song text, length_of_the_song float, PRIMARY KEY \
                 (session_Id,item_num_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)


                    

In [153]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line)
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_his_heard (session_Id, item_num_in_session, artist, song, length_of_the_song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
       
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        #print(session)

#### Do a SELECT to verify that the data have been inserted into each table

In [154]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = '''SELECT * FROM music_app_his_heard WHERE session_Id=139 and item_num_in_session=3'''
try:
    rows = session.execute(query)
    #print(rows)
except Exception as e:
    print(e)
for row in rows:
    print(rows)
    print(row.artist, row.song, row.length_of_the_song)


Mr Oizo Flat 55 144.03872680664062


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [155]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query ="CREATE TABLE IF NOT EXISTS music_app_user"
query = query + "(user_id int, item_num_in_session int, session_Id int, artist text, song text, user text, PRIMARY KEY \
                 ((user_id, session_Id), item_num_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        print(line)
        query = "INSERT INTO music_app_user (user_id, item_num_in_session, session_Id, artist, song, user)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
       
        session.execute(query, (int(line[10]), int(line[3]), int(line[8]), line[0], line[9], (line[1]+' '+line[4])))
        #print(session)

['Mynt', 'Celeste', 'F', '2', 'Williams', '166.94812', 'free', 'Klamath Falls, OR', '52', 'Playa Haters', '53']
['Taylor Swift', 'Celeste', 'F', '3', 'Williams', '230.47791', 'free', 'Klamath Falls, OR', '52', 'You Belong With Me', '53']
['Amy Winehouse', 'Celeste', 'F', '4', 'Williams', '229.85098', 'free', 'Klamath Falls, OR', '52', 'Valerie', '53']
['Jimmy Eat World', 'Celeste', 'F', '5', 'Williams', '285.83138', 'free', 'Klamath Falls, OR', '52', 'Dizzy', '53']
['Maldita Nerea', 'Anabelle', 'F', '0', 'Simpson', '241.162', 'free', 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', '158', 'Supelicula', '69']
['Fluke', 'Connar', 'M', '0', 'Moreno', '478.92853', 'free', 'Houston-The Woodlands-Sugar Land, TX', '168', 'Bermuda', '62']
['Habib KoitÃ\x83Â©', 'Jayden', 'M', '0', 'Fox', '285.1522', 'free', 'New Orleans-Metairie, LA', '185', 'Din Din Wo', '101']
['The Kooks', 'Sara', 'F', '0', 'Johnson', '132.25751', 'paid', 'Winston-Salem, NC', '152', "Eddie's Gun", '95']
['Blues Traveler', 'Sar

In [156]:
query = '''SELECT * FROM music_app_user WHERE user_id = 101 AND session_ID = 100 ORDER BY item_num_in_session'''

rows = session.execute(query)
for row in rows:
    print(row.artist, row.song , row.user)

Survivor Eye Of The Tiger Jayden Fox


In [157]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query ="CREATE TABLE IF NOT EXISTS music_app_who_listen"
query = query + "(song text, artist text, user text, user_id int, session_Id int, PRIMARY KEY \
                 (song, artist, user))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        print(line)
        query = "INSERT INTO music_app_who_listen (song, artist, user, user_id, session_Id)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
       
        session.execute(query, (line[9],line[0],(line[1]+' '+line[4]), int(line[10]), int(line[8])))
        print(session)
                    

['Mynt', 'Celeste', 'F', '2', 'Williams', '166.94812', 'free', 'Klamath Falls, OR', '52', 'Playa Haters', '53']
['Taylor Swift', 'Celeste', 'F', '3', 'Williams', '230.47791', 'free', 'Klamath Falls, OR', '52', 'You Belong With Me', '53']
['Amy Winehouse', 'Celeste', 'F', '4', 'Williams', '229.85098', 'free', 'Klamath Falls, OR', '52', 'Valerie', '53']
['Jimmy Eat World', 'Celeste', 'F', '5', 'Williams', '285.83138', 'free', 'Klamath Falls, OR', '52', 'Dizzy', '53']
['Maldita Nerea', 'Anabelle', 'F', '0', 'Simpson', '241.162', 'free', 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD', '158', 'Supelicula', '69']
['Fluke', 'Connar', 'M', '0', 'Moreno', '478.92853', 'free', 'Houston-The Woodlands-Sugar Land, TX', '168', 'Bermuda', '62']
['Habib KoitÃ\x83Â©', 'Jayden', 'M', '0', 'Fox', '285.1522', 'free', 'New Orleans-Metairie, LA', '185', 'Din Din Wo', '101']
['The Kooks', 'Sara', 'F', '0', 'Johnson', '132.25751', 'paid', 'Winston-Salem, NC', '152', "Eddie's Gun", '95']
['Blues Traveler', 'Sar

In [158]:
query = '''SELECT * FROM music_app_who_listen WHERE song = 'All Hands Against His Own' '''

rows = session.execute(query)
for row in rows:
    print(row.user)

Sara Johnson


### Drop the tables before closing out the sessions

In [159]:
## TO-DO: Drop the table before closing out the sessions

In [160]:
session.execute('DROP TABLE IF EXISTS music_app_his_heard')
session.execute('DROP TABLE IF EXISTS music_app_user')
session.execute('DROP TABLE IF EXISTS music_app_who_listen')

### Close the session and cluster connection¶

In [161]:
session.shutdown()
cluster.shutdown()